In [1]:
import pandas as pd
import numpy as np
import json
import altair as alt

as an easy first step, let's conpare June 2022 - the last date we have data for all home nations

In [2]:
eng_df = pd.read_excel('England/June-2022-AE-by-provider-dfTKt-revised-110523.xls', skiprows=15)
eng_df = eng_df[["Code", "STP", "Percentage in 4 hours or less (all)"]]
eng_df = eng_df.rename(columns={"Percentage in 4 hours or less (all)": "value", 'Code': 'id', 'STP': 'name'})
eng_df = eng_df.dropna()
eng_df['nation'] = 'England'

Wales

In [3]:
with open("Wales/4hr_ae_2022.json") as f:
    data = json.load(f)
    data = data['value']
    wls_df = pd.DataFrame(data)
wls_df = wls_df.query("Date_Code == '2022m06'")
wls_df = wls_df[wls_df.Hospital_ItemName_ENG.str.contains("Board")]
wls_df = wls_df[["Hospital_Code" ,"Hospital_ItemName_ENG", "Data"]]
wls_df = wls_df.rename(columns={"Hospital_Code": "id", "Hospital_ItemName_ENG": "name", "Data": "value"})
wls_df['value'] = wls_df['value'].astype(float)*0.01
wls_df['nation'] = 'Wales'

Northern Ireland

In [4]:
nir_df = pd.read_csv("Northern_Ireland/Emergency Care Waiting Times.csv")
nir_df = nir_df.query("Month == 'Jun 2022'")
nir_df['Total'] = nir_df['Total'].str.replace(',', '').astype(float)
nir_df['Under 4 Hours'] = nir_df['Under 4 Hours'].str.replace(',', '').astype(float)
nir_df = nir_df.groupby('Trust').sum()
nir_df['value'] = nir_df['Under 4 Hours'] / nir_df['Total']
nir_df = nir_df.reset_index()
nir_df = nir_df.rename(columns={"Trust": "name"})
nir_df['nation'] = 'Northern Ireland'
nir_df = nir_df[["name", "value", "nation"]]

Scotland

In [5]:
sct_df = pd.read_csv("Scotland/ae.csv")
sct_df = sct_df.query("Month == 202206")
#sct_df = sct_df.query("DepartmentType == 'Emergency Department'")

names_df = pd.read_csv("Scotland/Health_Boards_(May_2016)_Names_and_Codes_in_Scotland.csv")
# build into a "HB16CD" to "HB16NM" mapping
names_df = names_df[["HB16CD", "HB16NM"]]
names_df = names_df.rename(columns={"HB16CD": "HBT", "HB16NM": "name"})

sct_df = sct_df.merge(names_df, on="HBT")

sct_df = sct_df[["name", "HBT", "NumberOfAttendancesAll", "NumberWithin4HoursAll"]]
sct_df = sct_df.groupby(['HBT', 'name']).sum()

sct_df = sct_df.reset_index()
sct_df['value'] = sct_df['NumberWithin4HoursAll'] / sct_df['NumberOfAttendancesAll']

sct_df = sct_df[["HBT", "name", "value"]]
sct_df = sct_df.rename(columns={"HBT": "id"})
sct_df['nation'] = 'Scotland'

sct_df



,id,name,value,nation
0,S08000015,Ayrshire and Arran,0.690096,Scotland
1,S08000016,Borders,0.654734,Scotland
2,S08000017,Dumfries and Galloway,0.793596,Scotland
3,S08000019,Forth Valley,0.627218,Scotland
4,S08000020,Grampian,0.733617,Scotland
5,S08000022,Highland,0.849459,Scotland
6,S08000024,Lothian,0.659932,Scotland
7,S08000025,Orkney,0.926205,Scotland
8,S08000026,Shetland,0.947368,Scotland
9,S08000028,Western Isles,0.990895,Scotland


UK

In [6]:
df = pd.concat([eng_df, wls_df, nir_df, sct_df])
df = df.dropna(subset=["name", "value"])

df['nameShort'] = df['name'].str.replace("NHS ", "")
df['nameShort'] = df['nameShort'].str.replace("Health Board", "")
df['nameShort'] = df['nameShort'].str.replace("Integrated Care Board", "")

df = df.query("id != '-'")
df = df.sort_values(by=['value'], ascending=False)

In [8]:
df

,id,name,value,nation,nameShort
15,QNQ,NHS Frimley Integrated Care Board,1.000000,England,Frimley
255,7A7W11000024,Powys Teaching Health Board,0.998466,Wales,Powys Teaching
9,S08000028,Western Isles,0.990895,Scotland,Western Isles
8,S08000026,Shetland,0.947368,Scotland,Shetland
7,S08000025,Orkney,0.926205,Scotland,Orkney
...,...,...,...,...,...
3,NaN,Southern,0.560864,Northern Ireland,Southern
7,QUE,NHS Cambridgeshire And Peterborough Integrated...,0.536967,England,Cambridgeshire And Peterborough
1,NaN,Northern,0.516056,Northern Ireland,Northern
4,NaN,Western,0.508953,Northern Ireland,Western


In [9]:
df.query("nation == 'Northern Ireland'")

,id,name,value,nation,nameShort
2,NaN,South Eastern,0.565966,Northern Ireland,South Eastern
3,NaN,Southern,0.560864,Northern Ireland,Southern
1,NaN,Northern,0.516056,Northern Ireland,Northern
4,NaN,Western,0.508953,Northern Ireland,Western
0,NaN,Belfast,0.436201,Northern Ireland,Belfast


In [7]:
chart = alt.Chart(df.tail(10)).mark_bar().encode(
    y=alt.Y('nameShort:N', sort='x'),
    x=alt.X('value:Q', axis=alt.Axis(format='%')),
    color=alt.Color('nation:N', legend=alt.Legend(title="Nation")),
    tooltip=['name', 'value']
).properties(
    width=600,
    height=400
)

chart.display()

alt.Chart(...)

In [115]:
df.tail(20)

,id,name,value,nation,nameShort
27,QH8,NHS Mid And South Essex Integrated Care Board,0.681306,England,Mid And South Essex
280,7A2W11000025,Hywel Dda University Health Board,0.678879,Wales,Hywel Dda University
28,QMM,NHS Norfolk And Waveney Integrated Care Board,0.665442,England,Norfolk And Waveney
6,S08000024,Lothian,0.659932,Scotland,Lothian
12,QJK,NHS Devon Integrated Care Board,0.656094,England,Devon
1,S08000016,Borders,0.654734,Scotland,Borders
21,QM7,NHS Hertfordshire And West Essex Integrated Ca...,0.653668,England,Hertfordshire And West Essex
236,7A4W11000029,Cardiff and Vale University Health Board,0.639309,Wales,Cardiff and Vale University
20,QGH,NHS Herefordshire And Worcestershire Integrate...,0.638711,England,Herefordshire And Worcestershire
17,QOP,NHS Greater Manchester Integrated Care Board,0.630361,England,Greater Manchester


In [ ]:
alt.Chart

In [107]:
df.groupby('nation').mean()

/var/folders/07/g_yq2xsn0mvb4xnyvr18jjmw0000gn/T/ipykernel_43144/1760783516.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('nation').mean()


,value
nation,
England,0.723788
Northern Ireland,0.517608
Scotland,0.787312
Wales,0.731899


In [102]:
df.sort_values(by=['value'], ascending=False)
df

,id,name,value,nation
2,QOX,"NHS Bath And North East Somerset, Swindon And ...",0.728448,England
3,QHG,"NHS Bedfordshire, Luton And Milton Keynes Inte...",0.853728,England
4,QHL,NHS Birmingham And Solihull Integrated Care Board,0.696995,England
5,QUY,"NHS Bristol, North Somerset And South Gloucest...",0.692137,England
6,QU9,"NHS Buckinghamshire, Oxfordshire And Berkshire...",0.719976,England
...,...,...,...,...
5,S08000022,Highland,0.849459,Scotland
6,S08000024,Lothian,0.659932,Scotland
7,S08000025,Orkney,0.926205,Scotland
8,S08000026,Shetland,0.947368,Scotland


In [76]:
sct_df.to_excel("temp.xlsx")

In [75]:
sct_df.name.value_counts()

Highland                 4
Grampian                 3
Lothian                  3
Ayrshire and Arran       2
Dumfries and Galloway    2
Borders                  1
Forth Valley             1
Orkney                   1
Shetland                 1
Western Isles            1
Name: name, dtype: int64

In [68]:
sct_df.query("HBT == 'S08000028'")

,Month,Country,HBT,TreatmentLocation,DepartmentType,NumberOfAttendancesAll,NumberWithin4HoursAll,NumberOver4HoursAll,PercentageWithin4HoursAll,NumberOfAttendancesEpisode,...,PercentageWithin4HoursEpisode,PercentageWithin4HoursEpisodeQF,NumberOver8HoursEpisode,NumberOver8HoursEpisodeQF,PercentageOver8HoursEpisode,PercentageOver8HoursEpisodeQF,NumberOver12HoursEpisode,NumberOver12HoursEpisodeQF,PercentageOver12HoursEpisode,PercentageOver12HoursEpisodeQF
14335,202006,S92000003,S08000028,W107H,Emergency Department,374,368,6,98.4,374.0,...,98.4,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN


In [67]:
sct_df[sct_df.HBT.str.contains("S08000021")]

,Month,Country,HBT,TreatmentLocation,DepartmentType,NumberOfAttendancesAll,NumberWithin4HoursAll,NumberOver4HoursAll,PercentageWithin4HoursAll,NumberOfAttendancesEpisode,...,PercentageWithin4HoursEpisode,PercentageWithin4HoursEpisodeQF,NumberOver8HoursEpisode,NumberOver8HoursEpisodeQF,PercentageOver8HoursEpisode,PercentageOver8HoursEpisodeQF,NumberOver12HoursEpisode,NumberOver12HoursEpisodeQF,PercentageOver12HoursEpisode,PercentageOver12HoursEpisodeQF


In [63]:
sct_df.columns

Index(['Month', 'Country', 'HBT', 'TreatmentLocation', 'DepartmentType',
       'NumberOfAttendancesAll', 'NumberWithin4HoursAll',
       'NumberOver4HoursAll', 'PercentageWithin4HoursAll',
       'NumberOfAttendancesEpisode', 'NumberOfAttendancesEpisodeQF',
       'NumberWithin4HoursEpisode', 'NumberWithin4HoursEpisodeQF',
       'NumberOver4HoursEpisode', 'NumberOver4HoursEpisodeQF',
       'PercentageWithin4HoursEpisode', 'PercentageWithin4HoursEpisodeQF',
       'NumberOver8HoursEpisode', 'NumberOver8HoursEpisodeQF',
       'PercentageOver8HoursEpisode', 'PercentageOver8HoursEpisodeQF',
       'NumberOver12HoursEpisode', 'NumberOver12HoursEpisodeQF',
       'PercentageOver12HoursEpisode', 'PercentageOver12HoursEpisodeQF'],
      dtype='object')

,HBT,name
0,S08000015,Ayrshire and Arran
1,S08000016,Borders
2,S08000017,Dumfries and Galloway
3,S08000018,Fife
4,S08000019,Forth Valley
5,S08000020,Grampian
6,S08000021,Greater Glasgow and Clyde
7,S08000022,Highland
8,S08000023,Lanarkshire
9,S08000024,Lothian


In [71]:
names_df

,HB16CD,HB16NM,OBJECTID
0,S08000015,Ayrshire and Arran,0
1,S08000016,Borders,1
2,S08000017,Dumfries and Galloway,2
3,S08000018,Fife,3
4,S08000019,Forth Valley,4
5,S08000020,Grampian,5
6,S08000021,Greater Glasgow and Clyde,6
7,S08000022,Highland,7
8,S08000023,Lanarkshire,8
9,S08000024,Lothian,9


In [70]:
names_df.to_dict('index')

{0: {'HB16CD': 'S08000015', 'HB16NM': 'Ayrshire and Arran', 'OBJECTID': 0},
 1: {'HB16CD': 'S08000016', 'HB16NM': 'Borders', 'OBJECTID': 1},
 2: {'HB16CD': 'S08000017', 'HB16NM': 'Dumfries and Galloway', 'OBJECTID': 2},
 3: {'HB16CD': 'S08000018', 'HB16NM': 'Fife', 'OBJECTID': 3},
 4: {'HB16CD': 'S08000019', 'HB16NM': 'Forth Valley', 'OBJECTID': 4},
 5: {'HB16CD': 'S08000020', 'HB16NM': 'Grampian', 'OBJECTID': 5},
 6: {'HB16CD': 'S08000021',
  'HB16NM': 'Greater Glasgow and Clyde',
  'OBJECTID': 6},
 7: {'HB16CD': 'S08000022', 'HB16NM': 'Highland', 'OBJECTID': 7},
 8: {'HB16CD': 'S08000023', 'HB16NM': 'Lanarkshire', 'OBJECTID': 8},
 9: {'HB16CD': 'S08000024', 'HB16NM': 'Lothian', 'OBJECTID': 9},
 10: {'HB16CD': 'S08000025', 'HB16NM': 'Orkney', 'OBJECTID': 10},
 11: {'HB16CD': 'S08000026', 'HB16NM': 'Shetland', 'OBJECTID': 11},
 12: {'HB16CD': 'S08000027', 'HB16NM': 'Tayside', 'OBJECTID': 12},
 13: {'HB16CD': 'S08000028', 'HB16NM': 'Western Isles', 'OBJECTID': 13}}

In [48]:
nir_df.reset_index()

,Trust,Under 4 Hours,Total,value
0,Belfast,6550.0,15016.0,0.436201
1,Northern,6364.0,12332.0,0.516056
2,South Eastern,6675.0,11794.0,0.565966
3,Southern,7630.0,13604.0,0.560864
4,Western,5486.0,10779.0,0.508953


/var/folders/07/g_yq2xsn0mvb4xnyvr18jjmw0000gn/T/ipykernel_43144/3161006780.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  nir_df.groupby('Trust').sum()


,Under 4 Hours,Total
Trust,,
Belfast,6550.0,15016.0
Northern,6364.0,12332.0
South Eastern,6675.0,11794.0
Southern,7630.0,13604.0
Western,5486.0,10779.0


In [33]:
wls_df

,id,name,value,nation
236,7A4W11000029,Cardiff and Vale University Health Board,63.9309,Wales
237,7A6W11000028,Aneurin Bevan University Health Board,71.5414,Wales
244,7A1W11000023,Betsi Cadwaladr University Health Board,62.7427,Wales
255,7A7W11000024,Powys Teaching Health Board,99.8466,Wales
280,7A2W11000025,Hywel Dda University Health Board,67.8879,Wales
